In [ ]:
## This file produces data for the model to use


import oss
import json
from dotenv import load_dotenv

## Load Config
with open('config/videos.json') as config_file:
    videos = json.load(config_file)
with open('config/name_to_url.json') as config_file:
    name_to_url = json.load(config_file)

load_dotenv(dotenv_path=".env")

from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_KEY'),
)


In [ ]:
## Grab Transcripts and Clean
from utils.video import download_video
from utils.transcript import download_transcript
import nltk
import string
from nltk.corpus import stopwords

# download_video('https://www.youtube.com/watch?v=Xdhe-meS30A', 'data/videos/video.mp4')
for link, video_id, name in videos:
    download_transcript(video_id, f'data/transcripts/raw/{name}.txt')
    with open(f'data/transcripts/raw/{name}.txt', 'r') as file:
        with open(f'data/transcripts/clean/{name}.txt', 'w') as file_clean:
            for i,line in enumerate(file):

                line = line.split(',')[2]
                ## Remove newline
                line = line.replace('\n', ' ')
                if i != 0:
                    file_clean.write(line)

nltk.download('stopwords')
for link, video_id, name in videos:
    with open(f'data/transcripts/clean/{name}.txt', 'r') as file:
        with open(f'data/transcripts/processed/{name}.txt', 'w') as file_processed:
            text = file.read()
            text = text.lower()
            text = text.translate(str.maketrans('', '', string.punctuation))
            stop_words = set(stopwords.words('english'))
            filtered_text = [word for word in text.split() if word not in stop_words]

            filtered_text = ' '.join(filtered_text)

            file_processed.write(text)

In [ ]:
## Use data/prompt.txt as prompt. Use OpenAI gpt4 to generate a response. Write that response into oneshot.

name = "hashing"
prompt = ""

with open(f'data/prompts/prompt.txt', 'r') as file:
    prompt = file.read()
with open(f'data/transcripts/processed/{name}.txt', 'r') as file:
    prompt = prompt + file.read()

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    model="gpt-4"
)

In [ ]:
print(chat_completion.choices[0].message.content)